In [1]:
#@title Install Dependencies and Download Models

!pip install transformers datasets

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import LogitsProcessor, set_seed
import numpy as np
import datasets

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
device = 'cuda:0'
model.to(device)
model = model.eval()

In [3]:
# @title Download lab files

import sys

![ ! -d 'llm_lab' ] && git clone https://github.com/ethz-privsec/llm_lab.git
%cd llm_lab
!git pull https://github.com/ethz-privsec/llm_lab.git
%cd ..
if "llm_lab" not in sys.path:
  sys.path.append("llm_lab")

Cloning into 'llm_lab'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 51 (delta 18), reused 41 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (51/51), 366.78 KiB | 6.01 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/llm_lab
From https://github.com/ethz-privsec/llm_lab
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
/content


In [4]:
# @title Example of how to generate 100 tokens of text without watermarking

from llm_lab.gpt_generate import generate_with_seed, gen_red_list

prompt = "Boston is one of the oldest municipalities in America,"
print(generate_with_seed(model, tokenizer, prompt, seed=42))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Boston is one of the oldest municipalities in America, but it's also among those with a history that dates back to at least 1857. The city has been home for most Aryan settlers from California until they settled here around 1787 when Columbus' New World arrived on Endurance Island and secured their land close by (the site was known as "Ferry Rock"). In other words: Kimball County doesn't exist right now — just be sure you're aware of its existence…
 [ Read more ] Thom Ann / Fox News $64.99


You will now implement three different watermarking schemes:
1. A simple scheme that never outputs the letter 'e' (lowercase or uppercase)
2. A red-list scheme, that generates a random list of banned tokens for each token generation.
3. A soft red-list scheme, that also generates a random red-list, but just biases the LLM against these tokens instead of outright banning them, by substracting the value `logit_offset=2` from the logits of each red-listed token.

You should implement each of these schemes as a `LogitsProcessor` class.

For the red-list schemes, you should use `gpt_generate.gen_red_list` to generate a red list containing 50% of the LLM's vocabulary.
The seed for generating the pseudorandom red list is computed from the previous token processed by the model.

So for example, if the model has so far processed the string "my name is " (which tokenizes as `[1820, 1438, 318, 220]`), then the red list for the next token to be generated is `**gen_red_list(torch.LongTensor([220]), model.config.vocab_size)** = [43383,  7006, 40846, ...]`.

In [62]:
#@title Exercise 1: Implement a trivial watermarking scheme that samples text without any 'e' (lowercase or uppercase)

class NoEsLogitsProcessor(LogitsProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        tokens = tokenizer.convert_ids_to_tokens(list(range(model.config.vocab_size)))
        e_tokens = [t for t in tokens if 'e' in t.lower()]
        self.e_ids = tokenizer.convert_tokens_to_ids(e_tokens)

    def __call__(self, input_ids, scores):
        """
        Processes the output scores of the LLM before generating the next token.
        Args:
            input_ids: torch.LongTensor of shape (batch_size, sequence_length) — Indices of input sequence tokens in the vocabulary.
            scores: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — Logits for the next token to be generated.
        Returns: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — The processed logits.
        """
        scores[:, self.e_ids] = -float('inf')
        return scores


no_e_processor = NoEsLogitsProcessor()

prompt = "Anton Vowl is missing. Ransacking his Paris flat, a group of his faithful companions trawl through his diary for any hint as to his location and, insidiously, a ghost, from Vowl's past starts to cast its malignant shadow.\n "
output = generate_with_seed(model, tokenizer, prompt, logits_processor=no_e_processor, seed=42)
print(output)

Anton Vowl is missing. Ransacking his Paris flat, a group of his faithful companions trawl through his diary for any hint as to his location and, insidiously, a ghost, from Vowl's past starts to cast its malignant shadow.
  In fact that protagonist who was in such agony at first sight looks nothing but an ill-lucard son on all fours — sadistic bastard? A coward (and possibly also unkind) *cough* Mr N'Vow! Finally coming into contact with both Jonsonakos ("Christina Tyngrav") whom will do anything Ali affords him if it suits us; or two kilograms wolfish right off B1226 "Bitch" Gokai: about


In [103]:
#@title Exercise 2: Implement a red-list watermarking scheme

class RedListLogitsProcessor(LogitsProcessor):
    def __init__(self, red_frac=0.5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.red_frac = red_frac

    def __call__(self, input_ids, scores):
        """
        Processes the output scores of the LLM before generating the next token.
        Args:
            input_ids: torch.LongTensor of shape (batch_size, sequence_length) — Indices of input sequence tokens in the vocabulary.
            scores: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — Logits for the next token to be generated.
        Returns: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — The processed logits.
        """
        red_ids = np.array([gen_red_list(b_input[-1], model.config.vocab_size, frac_red=self.red_frac) for b_input in input_ids])
        scores[:, red_ids] = -float('inf')
        return scores

red_list_processor = RedListLogitsProcessor()

prompt = "Boston is one of the oldest municipalities in America,"
output = generate_with_seed(model, tokenizer, prompt, logits_processor=red_list_processor, seed=42)
print(output)

Boston is one of the oldest municipalities in America, but it's not a spiritual successor. It had been created after former Premier Harry Marnelli drew up legislation on behalf (in his first term as mayor) and appointed two new members: Daniel Blakeley Ryves from Endurance Township -- who served five terms earlier this year under current Mayor Philip Ayroyama — Michael Rieckenham Jr.. The township voted for Yang last week when an election was held against state ballot measures to dissolve public school systems because they have problems or defects with charter dogs


In [72]:
#@title Exercise 3: Implement a soft red-list watermarking scheme

class SoftRedListLogitsProcessor(LogitsProcessor):
    def __init__(self, red_frac=0.5, logit_offset=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.red_frac = red_frac
        self.logit_offset = logit_offset

    def __call__(self, input_ids, scores):
        """
        Processes the output scores of the LLM before generating the next token.
        Args:
            input_ids: torch.LongTensor of shape (batch_size, sequence_length) — Indices of input sequence tokens in the vocabulary.
            scores: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — Logits for the next token to be generated.
        Returns: torch.FloatTensor of shape (batch_size, model.config.vocab_size) — The processed logits.
        """
        red_ids = np.array([gen_red_list(b_input[-1], model.config.vocab_size, frac_red=self.red_frac) for b_input in input_ids])
        scores[:, red_ids] -= self.logit_offset
        return scores

soft_red_list_processor = SoftRedListLogitsProcessor()

prompt = "Boston is one of the oldest municipalities in America,"
output = generate_with_seed(model, tokenizer, prompt, logits_processor=soft_red_list_processor, seed=42)
print(output)

Boston is one of the oldest municipalities in America, but it's not a spiritual successor. It had been created after former Premier Harry Mudd and his family fled Sudan on Aug 13 2001 following atrocities by rebels there which left around 700 people dead including children under 18 years old who died at an open fire triggered earlier this year when Russian forces attacked rebel-controlled areas along military lines near Benghazi.
The elected members are from Sikh communities (with four more than LDS leaders) working alongside African Christians seeking local leadership to support both religious groups as well As for


Okay! We're now ready to start generating watermarked text.
We give you 20 prompts in `data/watermark_prompts.txt`.
For each of these, generate 100 more tokens using each of the three watermarking schemes, and save all of this as a numpy array for submission.

MAKE SURE TO USE `seed=42` FOR ALL YOUR GENERATIONS.

In [8]:
from tqdm import trange

with open('llm_lab/data/watermark_prompts.txt') as f:
  prompts = f.read().splitlines()

processors = [no_e_processor, red_list_processor, soft_red_list_processor]
outputs = []

seed = 42  # DON'T CHANGE THIS!!!

for i in trange(20):
  min_new_tokens = 100
  max_new_tokens = min_new_tokens

  for j in range(3):
    output = generate_with_seed(model, tokenizer, prompts[i], logits_processor=processors[j],
                                min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens, seed=seed)
    outputs.append(output)

print(outputs)
np.save("Q1_gens.npy", outputs)

100%|██████████| 20/20 [02:09<00:00,  6.49s/it]

['Perhaps one did not want to be loved so much as to be understood.\n\n\n* * **From "Dunwich\'s Book of Poisons and Torturing" by Dr Arthur Aunty (1875-1921)** ` That was my class on witchcraft in 1835, from which I will pass a bit into that book.\' - John Allyson This is an account about poulticing for pain: it consists primarily at affording unsanitary conditions such plants or animals can afford; i think this might also apply if all animal husbandry could possibly work', "Perhaps one did not want to be loved so much as to be understood. It was only that protagonist's weakness, however astonishing and frightening in nature it may have sounded within his own mind when described (which is what the reader might expect of a brilliant character classed with all those other superb qualities) could spring into open question whether such an openly machismoistic man really knew or believed he had earned this title? In short: I've managed neither — but there are more than enough examples relea

For the final part of this question, you now have to try and detect watermarked text.
We give you 80 pieces of text in `data/watermarked_gens.npy`.
For each piece of text you have to guess whether it was generated with:

1.   No watermark
2.   The dummy "no E's" watermark
3.   The red-list watermark
4.   The soft red-list watermark

We use the same `generate_with_seed` and `gen_red_list` implementations as you. Our red-list watermarking scheme also uses the same parameters (i.e., 50% of the tokens are red-listed, and for the soft version we substract 2.0 from the logits).

Exactly 20 of the 80 texts are generated with each of the 4 options above. Each text is comprised of a short prompt, followed by 100-200 generated tokens.

Store your guesses (1,2,3,4) for each piece of text in a numpy array.

In [117]:
outputs_secret = np.load("llm_lab/data/watermarked_gens.npy", allow_pickle=True)
assert len(outputs_secret) == 80

my_guesses = []
tokenized_output = tokenizer(list(outputs_secret))['input_ids']

for output in tokenized_output:
    red_count = 0
    e_count = False
    for i, t in enumerate(output[:-1]):
        red_list = gen_red_list(t, model.config.vocab_size, frac_red=.5)
        red_count += output[i+1] in red_list
        e_count += 'e' in tokenizer.convert_ids_to_tokens(t).lower()

    if red_count/len(output) <= .1:
        my_guesses.append(3)
    elif e_count/len(output) <= .07:
        my_guesses.append(2)
    elif red_count/len(output) <= 0.4:
        my_guesses.append(4)
    else:
        my_guesses.append(1)

assert len(my_guesses) == 80
np.save("Q1_guesses.npy", np.asarray(my_guesses))

In [118]:
my_guesses.count(1), my_guesses.count(2), my_guesses.count(3), my_guesses.count(4)

(20, 20, 20, 20)